In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html
Using TensorFlow backend.


In [2]:
def get_notes():
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing ",file)

        notes_to_parse = None

        try: 
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: 
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


In [3]:
def prepare_sequences(notes, n_vocab):
    
    sequence_length = 100

    pitchnames = sorted(set(item for item in notes))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # Normalization
    network_input = network_input / float(n_vocab)
    # One-hot encoding
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [4]:
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [6]:
def train(model, network_input, network_output):
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=16, callbacks=callbacks_list)

In [7]:
def train_network():
    notes = get_notes()

    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [8]:
train_network()

Parsing midi_songs\2ndMovementOfSinisterFootwear.mid
Parsing midi_songs\55Dive.mid
Parsing midi_songs\5To10.mid
Parsing midi_songs\634-5789.mid
Parsing midi_songs\914.mid
Parsing midi_songs\ABC.mid
Parsing midi_songs\ACertainSmile.mid
Parsing midi_songs\ACrush.mid
Parsing midi_songs\Adayinalifeofafool.mid
Parsing midi_songs\Adventage.mid
Parsing midi_songs\AffairInSanMiguel.mid
Parsing midi_songs\AFifthofBeethoven.mid
Parsing midi_songs\Afoggydayilondontown.mid
Parsing midi_songs\AfterTheLoveHasGone.mid
Parsing midi_songs\AfterTheRainHasFallen.mid
Parsing midi_songs\AgeOfAquarius.mid
Parsing midi_songs\Aintchagottired.mid
Parsing midi_songs\AintNothingLikeRealThingBaby.mid
Parsing midi_songs\AintNoWomanLikeTheOneIGot.mid
Parsing midi_songs\Aja.mid
Epoch 1/10
17511/17511 [==============================] - 923s 53ms/step - loss: 5.2420
Epoch 2/10
17511/17511 [==============================] - 896s 51ms/step - loss: 4.8452
Epoch 3/10
17511/17511 [==============================] - 895s 51m